In [5]:
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
import numpy as np

Using TensorFlow backend.


In [6]:
(train_data,train_target),(test_data,test_target)=boston_housing.load_data()

In [7]:
train_data.shape

(404, 13)

In [8]:
test_data.shape

(102, 13)

In [9]:
train_target

array([ 15.2,  42.3,  50. ,  21.1,  17.7,  18.5,  11.3,  15.6,  15.6,
        14.4,  12.1,  17.9,  23.1,  19.9,  15.7,   8.8,  50. ,  22.5,
        24.1,  27.5,  10.9,  30.8,  32.9,  24. ,  18.5,  13.3,  22.9,
        34.7,  16.6,  17.5,  22.3,  16.1,  14.9,  23.1,  34.9,  25. ,
        13.9,  13.1,  20.4,  20. ,  15.2,  24.7,  22.2,  16.7,  12.7,
        15.6,  18.4,  21. ,  30.1,  15.1,  18.7,   9.6,  31.5,  24.8,
        19.1,  22. ,  14.5,  11. ,  32. ,  29.4,  20.3,  24.4,  14.6,
        19.5,  14.1,  14.3,  15.6,  10.5,   6.3,  19.3,  19.3,  13.4,
        36.4,  17.8,  13.5,  16.5,   8.3,  14.3,  16. ,  13.4,  28.6,
        43.5,  20.2,  22. ,  23. ,  20.7,  12.5,  48.5,  14.6,  13.4,
        23.7,  50. ,  21.7,  39.8,  38.7,  22.2,  34.9,  22.5,  31.1,
        28.7,  46. ,  41.7,  21. ,  26.6,  15. ,  24.4,  13.3,  21.2,
        11.7,  21.7,  19.4,  50. ,  22.8,  19.7,  24.7,  36.2,  14.2,
        18.9,  18.3,  20.6,  24.6,  18.2,   8.7,  44. ,  10.4,  13.2,
        21.2,  37. ,

标准化：$X_t=\frac{X-mean}{std}$

In [10]:
mean=train_data.mean(axis=0)
std=train_data.std(axis=0)
trian_data=(train_data-mean)/std
test_data=(test_data-mean)/std

In [11]:
def build_model():
    model=Sequential()
    # 64 > 13 : 层里的大小 要大于 特征数 先将维度放大, 在高维空间中抽取它的特征
    model.add(Dense(64,input_shape=(13,),activation="relu"))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(1))
    model.compile(optimizer=RMSprop(),loss="mse", metrics=['mae'])
    return model

<img src="imgs/fold_cross_validation.jpg">

In [12]:
k=4
num_val_samples=len(train_data)//k
num_epochs=100
all_scores=[]


In [ ]:
for i in range(k):
    val_data=train_data[i*num_val_samples:(i+1)*num_val_samples]
    val_targets=train_target[i*num_val_samples:(i+1)*num_val_samples]
    
    #np.concatenate 连接函数
    partial_train_data=np.concatenate([
        train_data[ : i*num_val_samples],
        train_data[(i+1)*num_val_samples: ]],
        axis=0)
    partial_train_targets=np.concatenate([
        train_target[ : i*num_val_samples],
        train_target[(i+1)*num_val_samples: ]],
        axis=0)
    model=build_model()
    model.fit(partial_train_data,partial_train_targets,
             epochs=num_epochs,batch_size=1,verbose=False)
    
    val_mse,val_mae=model.evaluate(val_data,val_targets,verbose=False)
    
    all_scores.append(val_mae)
    print("第",i+1,"折，MSE:",val_mse,"MAE:",val_mae)

In [17]:
np.mean(all_scores)

3.2984534024995544

In [18]:
model=build_model()
model.fit(partial_train_data,partial_train_targets,epochs=80,batch_size=16,verbose=False)

In [19]:
test_mse,test_mae=model.evaluate(train_data,train_target)

404/404 [==============================] - 0s 638us/step


In [20]:
test_mse

29.186944168392973